# NYC MoMA Artworks - Collections In-take and Space-use Forecasting

## Table of Contents
<ol>
  <li><a href="#load-libraries">Load libraries</a></li>
  <li><a href="#data-process">Data Preprocessing</a></li>
  <li><a href="#ts-decomp">Overall Time Series Decomposition</a></li>
  <li><a href="#spatial">Spatial Time Series Analysis</a></li>
  <ol>
    <li><a href="#spatial-ts">Analysis of Space-use over Time</a></li>
  </ol>
  <li><a href="#ts-model-declare">Time Series Model Initialization</a></li>
  <li><a href="#ts-model">Time Series Model Comparisons</a></li>
  <ol>
    <li><a href="#ts-model-tv">Time Series Model Training & Validation</a></li>
    <li><a href="#ts-model-forecast">Time Series Forecasting Estimates</a></li>
  </ol>
  <li><a href="#conclusions">Conclusions</a></li>
  <li><a href="#recommendations">Recommendations</a></li>
</ol>

<section id="load-libraries">
    <h2>Load Libraries</h2>
</section>

In [1]:
import pandas as pd
import math
import seaborn as sns
from darts import TimeSeries
from darts.models import NaiveDrift, NaiveMovingAverage, ExponentialSmoothing, Croston
from darts.statistics import check_seasonality, stationarity_tests, plot_residuals_analysis
import statsmodels.api as sm
from sklearn import TimeSeriesSplit

In [2]:
# Define Necessary Functions
def gen_ts_features(df, date_column):
    # Generates Time Series Features based on date_column
    # Makes for easier analysis of operational (day-to-day) processes.
    # df["date_column"] = pd.to_datetime(df[date_column])
    df["day_of_week"] = df[date_column].dt.day_name()
    df["week_of_month"] = (df.date_column.day / 7).apply(lambda x: math.ceil(x))
    df["week_of_year"] = df[date_column].dt.isocalendar().week
    df["month_of_year"] = df[date_column].dt.month_name()
    df["year"] = df[date_column].dt.isocalendar().year
    return df

def is_stationary(series):
    # Hypothesis Testing for Time Series data
    # Perform two separate stationarity tests
    result = stationarity_tests(series, p_value_threshold_adfuller=0.05, p_value_threshold_kpss=0.05)

    if result:
        print("The time series is stationary.")
    else:
        print("The time series is non-stationary.")
    return result

def has_seasonality(series):
    result = check_seasonality(series)

    if result[0]:
        print(f"The time series has seaonality, with a period of {result[1]}")
    else:
        print("The time series does not have seasonality.")
    return result

In [ ]:
# Data Preprocessing

# Sample data (replace with your actual data)
X = pd.DataFrame(np.random.randn(100, 10))  # Features
y = np.random.randn(100)  # Target

# Sort data by date column

# spatial_totol_series
# accession_counts_series

In [ ]:
# Spatial Analysis
# Plot with trend lines for each category
sns.lmplot(x = "obj_num", y = "spatial_total", hue = "category", data = data)

In [ ]:
# Plot with trend lines for each category
sns.lmplot(x = "obj_num", y = "acc_sum", hue = "category", data = data)

In [ ]:
# The next two steps are operative for knowing which models apply
# to the domain our data fits into (read: intermittent demand forecasting)

In [3]:
# Overall TS Decomposition

In [ ]:
# Weekly Rolling Average

In [ ]:
# Monthly Rolling Average

In [ ]:
# Time Series Statistical Tests
is_stationary(series)

In [ ]:
has_seasonality(series)

In [ ]:
# Time Series Model Comparison Setup
models = {
    # Baseline Statistical Models
    "Naive": NaiveDrift(),
    "Moving Average": NaiveMovingAverage(),
    "Exponential Smoothing": ExponentialSmoothing(),
    
    # Domain Specific Models - Intermittent Demand Forecasting
    "CROSTON Method": Croston(version = "classic"),
    "SBA": Croston(version = "sba"), # Syntetos-Boylan Approximation
    "TSB": Croston(version = "tsb") # Teunter-Syntetos-Babai
}

In [ ]:
# Comparing Models for Selection
results = {
        "Model": [],
        "Forecast Steps": [],
        "MAE": [],
        "MSE": [],
        "RMSE": [],
        "Coverage Probability": [],
        "Predictions": [],
}

# How many periods to forecast out to
forecast_windows = [5, 10, 15, 20, 30]

# Splits data for cross-validation
tscv = TimeSeriesSplit(n_splits = 5)

# Estimating forecasts using historical data
for model_name, model in models.items():
    # Splits data using cross-validation
    for train_index, test_index in tscv.split(series):
        train, test = series[train_index], series[test_index]
        model.fit(train)
        
        # Create predictions using one model 
        # over various numbers of steps out.
        # Adjusts data-split accordingly.
        for step_count in forecast_windows:
            # To make sure that models forecast to their horizons honestly
            if len(test) < step_count:
                print(f"Skipping model {model_name} for forecast horizon {step_count} due to insufficient amounts of test data.")
                continue
            
            predictions = model.predict(steps = step_count)
            y_true = test[:step_count].values()
        
            # Metrics selected for model selection
            mae = mean_absolute_error(y_true, predictions)
            mse = mean_squared_error(y_true, predictions)
            # TODO: mase
            # TODO: r2_score
            rmse = np.sqrt(mean_squared_error(y_true, predictions))
            
            # Calculate Coverage Probability (assuming a 95% prediction interval)
            z_score = 1.96
            lower_bound = predictions - (z_score * np.std(predictions))
            upper_bound = predictions + (z_score * np.std(predictions))
            coverage_probability = np.mean((y_true >= lower_bound) & (y_true <= upper_bound))

            # Store all results
            results["Model"].append(model_name)
            results["Forecast Steps"].append(step_count)
            results["MAE"].append(mae)
            results["MSE"].append(mse)
            results["RMSE"].append(rmse)
            results["Coverage Probability"].append(coverage_probability)
            results["Predictions"].append(predictions.values())

In [ ]:
# Model performance visual comparisons

Potential Future Features:
- [ ] Hierarchical Modeling (e.g. by Department, by Credit, etc.)
- [ ] Popular ML models (ARIMA, RandomForest, MCMC, etc.)
- [ ] With Multi-variate Models, we can have SHAP feature importance explanation.
- [ ] Calculating Backlog durations.

# We're about the process!
If streamlining means less people work, we want less of it. Data belong to the social.